In [12]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching et_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s et_Scene"

In [86]:
# Array mit Eye-Tracking Daten ([x, y, t])
et_array = np.array([
    [-6, -3, 200],
    [-5, 1, 400],
    [-4, 0, 600],
    [-3, 2, 200],
    [-2, -1, 300],
])

bin_count = 16

et_pos_array = np.zeros(np.shape(et_array))
et_pos_array[:,:2] = et_array[:,:2]
et_time_array = et_array[:,2] / 400

# calculates angle of vector v
def polar_angle(v):
    return np.arctan(v[1]/v[0])


# takes polar angle and returns bin indey and binned angle
def bin_angle(angle):
    bins = np.linspace(0, 2*PI, bin_count+1)
    for i, bin in enumerate(bins):
        if bin > angle - PI/bin_count:
            return i, bin

print(np.linspace(0, 2*PI, bin_count+1))
bin_angle(PI)

[0.         0.39269908 0.78539816 1.17809725 1.57079633 1.96349541
 2.35619449 2.74889357 3.14159265 3.53429174 3.92699082 4.3196899
 4.71238898 5.10508806 5.49778714 5.89048623 6.28318531]


(8, 3.141592653589793)

In [89]:
%%capture_video --path "animations/eye_tracking.mp4"
%%manim -ql --fps 20 $video_scene

class et_Scene(Scene):
    def construct(self):
        plot_center = 4*RIGHT

        polarplane_pi = PolarPlane(
            azimuth_units = "degrees",
            size = 4,
            azimuth_label_font_size = 33.6,
            azimuth_step = bin_count,
            azimuth_label_buff = 0.2,
        ).add_coordinates(r_values= []).shift(plot_center)
        self.add(polarplane_pi)

        phi_vals = np.linspace(0, 2*PI, bin_count+1)
        r_vals = np.zeros_like(phi_vals)

        #graph = polarplane_pi.plot_line_graph(x_values = x_vals, y_values = y_vals)
        for i, pos in enumerate(et_pos_array):
            if i == 0:
                dot = Dot(color = WHITE, radius = 0.1).move_to(pos)
                #graph0 = polarplane_pi.plot_line_graph(x_values = [r_vals[i1] * np.cos(phi_vals[i1]) for i1 in len(phi_vals)], y_values = [r_vals[i1] * np.sin(phi_vals[i1]) for i1 in len(phi_vals)])
                #self.add(graph0)
                self.play(FadeIn(dot))
                self.wait(et_time_array[i])
            else:
                dot = Dot(color = WHITE, radius = 0.1).move_to(pos)
                arrow = Arrow(start = et_pos_array[i-1], end = pos, color = RED, stroke_width = 1, buff = 0.2, tip_length = 0.15)
                index, binned_angel = bin_angle(polar_angle(pos - et_pos_array[i-1]))
                r_vals[index] += 1
                print(index)
                plot_arrow =Arrow(start = [0, 0, 0], end = [r_vals[index] * np.cos(binned_angel), r_vals[index] * np.sin(binned_angel), 0], color = RED, buff = 0).shift(plot_center)
                #graph = polarplane_pi.plot_line_graph(x_values = [r_vals[i1] * np.cos(phi_vals[i1]) for i1 in len(phi_vals)], y_values = [r_vals[i1] * np.sin(phi_vals[i1]) for i1 in len(phi_vals)])
                self.play(FadeIn(dot), Create(arrow))
                self.play(TransformFromCopy(arrow, plot_arrow))#, ReplacementTransform(graph0, graph))
                self.wait(et_time_array[i])

3
0
3
0
